In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import math
import ast
import operator

# ------------------------------------------
# SAFE SCIENTIFIC EVALUATOR
# ------------------------------------------

ALLOWED_OPERATORS = {
    ast.Add: operator.add,
    ast.Sub: operator.sub,
    ast.Mult: operator.mul,
    ast.Div: operator.truediv,
    ast.Pow: operator.pow,
    ast.Mod: operator.mod,
}

ALLOWED_UNARY = {
    ast.UAdd: operator.pos,
    ast.USub: operator.neg
}

SAFE_FUNCTIONS = {
    "sin": lambda x: math.sin(math.radians(x)),
    "cos": lambda x: math.cos(math.radians(x)),
    "tan": lambda x: math.tan(math.radians(x)),
    "log": lambda x: math.log(x),
    "log10": lambda x: math.log10(x),
    "sqrt": lambda x: math.sqrt(x),
}

def safe_eval(expr):
    expr = expr.replace("^", "**")
    try:
        tree = ast.parse(expr, mode="eval")
    except Exception:
        raise ValueError("Invalid Expression")

    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)

        if isinstance(node, ast.Constant):
            return node.value

        if isinstance(node, ast.UnaryOp):
            val = _eval(node.operand)
            if type(node.op) in ALLOWED_UNARY:
                return ALLOWED_UNARY[type(node.op)](val)
            raise ValueError("Invalid unary op")

        if isinstance(node, ast.BinOp):
            left = _eval(node.left)
            right = _eval(node.right)
            if type(node.op) in ALLOWED_OPERATORS:
                return ALLOWED_OPERATORS[type(node.op)](left, right)
            raise ValueError("Invalid operator")

        if isinstance(node, ast.Call):
            func = node.func.id
            if func in SAFE_FUNCTIONS:
                val = _eval(node.args[0])
                return SAFE_FUNCTIONS[func](val)
            raise ValueError("Invalid function")

        raise ValueError("Invalid Expression")

    return _eval(tree)


# ------------------------------------------
# MAIN CALCULATOR CLASS
# ------------------------------------------

class Calculator(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Modern Scientific Calculator + History Panel")
        self.geometry("750x650")
        self.configure(bg="#111827")

        self.current_theme = "dark"
        self.history = []

        self._set_theme()
        self._build_display()
        self._build_buttons()
        self._build_history_panel()

        self.bind("<Return>", lambda e: self.calculate())
        self.bind("<BackSpace>", lambda e: self.backspace())
        self.bind_all("<Key>", self._on_keypress)

    # -----------------------------------
    # THEMES
    # -----------------------------------

    def _set_theme(self):
        if self.current_theme == "dark":
            self.bg = "#111827"
            self.btn_bg = "#1f2937"
            self.btn_fg = "white"
            self.accent = "#06b6d4"
            self.panel_bg = "#1e293b"
        else:
            self.bg = "#f3f4f6"
            self.btn_bg = "#e5e7eb"
            self.btn_fg = "black"
            self.accent = "#2563eb"
            self.panel_bg = "#ffffff"

        self.configure(bg=self.bg)

    def toggle_theme(self):
        self.current_theme = "light" if self.current_theme == "dark" else "dark"
        self._set_theme()
        self._repaint_buttons()
        self._repaint_history()

    # -----------------------------------
    # DISPLAY
    # -----------------------------------

    def _build_display(self):
        frame = tk.Frame(self, bg=self.bg)
        frame.pack(fill="x", pady=10)

        self.expr = tk.StringVar()
        self.res = tk.StringVar()

        self.entry = tk.Entry(frame, textvariable=self.expr,
                              font=("Segoe UI", 26),
                              bd=0, justify="right",
                              fg=self.accent, bg=self.bg)
        self.entry.pack(fill="x", padx=10, ipady=15)

        res_lbl = tk.Label(frame, textvariable=self.res,
                           font=("Segoe UI", 18),
                           fg=self.accent, bg=self.bg,
                           anchor="e")
        res_lbl.pack(fill="x", padx=10)

    # -----------------------------------
    # BUTTONS
    # -----------------------------------

    def _build_buttons(self):
        frame = tk.Frame(self, bg=self.bg)
        frame.pack(side="left", expand=True, fill="both", padx=10)

        buttons = [
            ["sin", "cos", "tan", "log"],
            ["log10", "sqrt", "(", ")"],
            ["7", "8", "9", "/"],
            ["4", "5", "6", "*"],
            ["1", "2", "3", "-"],
            ["0", ".", "%", "+"],
            ["AC", "⌫", "^", "="],
            ["Theme", "", "", "History"],
        ]

        self.btn_refs = []

        for r, row in enumerate(buttons):
            for c, text in enumerate(row):
                if not text:
                    continue

                btn = tk.Button(frame,
                                text=text,
                                font=("Segoe UI", 16),
                                bg=self.btn_bg,
                                fg=self.btn_fg,
                                activebackground="#374151",
                                bd=0,
                                command=lambda t=text: self._on_button(t))
                btn.grid(row=r, column=c, sticky="nsew", padx=5, pady=5, ipadx=5, ipady=12)
                self.btn_refs.append(btn)

        for i in range(len(buttons)):
            frame.rowconfigure(i, weight=1)
        for j in range(4):
            frame.columnconfigure(j, weight=1)

    # repaint on theme changes
    def _repaint_buttons(self):
        for b in self.btn_refs:
            b.configure(bg=self.btn_bg, fg=self.btn_fg)
        self.entry.configure(bg=self.bg, fg=self.accent)

    # -----------------------------------
    # BUTTON LOGIC
    # -----------------------------------

    def _on_button(self, t):
        if t == "=":
            self.calculate()
        elif t == "AC":
            self.expr.set("")
            self.res.set("")
        elif t == "⌫":
            self.backspace()
        elif t == "Theme":
            self.toggle_theme()
        elif t == "History":
            self.toggle_history_panel()
        else:
            self.expr.set(self.expr.get() + t)

    def backspace(self):
        self.expr.set(self.expr.get()[:-1])

    def calculate(self):
        try:
            out = safe_eval(self.expr.get())
            if isinstance(out, float) and out.is_integer():
                out = int(out)
            self.res.set(str(out))
            self._add_history(self.expr.get(), out)
        except Exception:
            self.res.set("Error")

    # -----------------------------------
    # KEYBOARD INPUT
    # -----------------------------------

    def _on_keypress(self, e):
        if e.char in "0123456789+-*/().%^":
            self.expr.set(self.expr.get() + e.char)
        elif e.char == "=":
            self.calculate()

    # -----------------------------------
    # HISTORY PANEL
    # -----------------------------------

    def _build_history_panel(self):
        self.history_frame = tk.Frame(self, bg=self.panel_bg, width=250)
        self.history_frame.pack(side="right", fill="y")
        self.history_frame.pack_forget()

        title = tk.Label(self.history_frame, text="History",
                         font=("Segoe UI", 14, "bold"),
                         bg=self.panel_bg, fg=self.btn_fg)
        title.pack(pady=10)

        self.hist_list = tk.Listbox(self.history_frame, bg=self.panel_bg,
                                    fg=self.btn_fg, font=("Segoe UI", 12),
                                    activestyle='none')
        self.hist_list.pack(fill="both", expand=True, padx=10)

        self.hist_list.bind("<Double-Button-1>", self._use_history)

        clr = tk.Button(self.history_frame, text="Clear History",
                        font=("Segoe UI", 12),
                        command=self._clear_history,
                        bg=self.btn_bg, fg=self.btn_fg, bd=0)
        clr.pack(pady=10)

    def toggle_history_panel(self):
        if self.history_frame.winfo_ismapped():
            self.history_frame.pack_forget()
        else:
            self.history_frame.pack(side="right", fill="y")
            self._repaint_history()

    def _add_history(self, expr, result):
        item = f"{expr} = {result}"
        self.history.insert(0, item)
        self.history = self.history[:50]  # limit
        self._reload_history()

    def _reload_history(self):
        self.hist_list.delete(0, tk.END)
        for h in self.history:
            self.hist_list.insert(tk.END, h)

    def _use_history(self, event):
        sel = self.hist_list.curselection()
        if not sel:
            return
        entry = self.hist_list.get(sel[0])
        expr = entry.split(" = ")[0]
        self.expr.set(expr)

    def _clear_history(self):
        self.history = []
        self._reload_history()

    def _repaint_history(self):
        self.history_frame.configure(bg=self.panel_bg)
        self.hist_list.configure(bg=self.panel_bg, fg=self.btn_fg)

if __name__ == "__main__":
    app = Calculator()
    app.mainloop()
